# Radiating electric field
Given by the equation
$$
\vec{E}_{rad}=\frac{1}{4\pi\epsilon_0}\left(\frac{q\hat{R}\times[(\hat{R}-\vec{\beta})\times\dot{\vec{\beta}}]}{c(1-\hat{R}\cdot\vec{\beta})^3R}\right)_{\text{retarded}},
$$
where $\vec{R}$ is the vector from the source to the field point
$$
\vec{R} = \vec{r} - \vec{r}',
$$
and where the time-dependent terms are evaluated at retarded time
$$
t' = t - \frac{R}{c}.
$$

In [4]:
import numpy as np
from vectorfields import CustomUV2D
from sys import argv
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


c = 299792458
m = 6.692e-27
q = 2 * 1.60217662e-19
e0 = 8.85418782e-12
k = -q**2/(4*np.pi*m*e0)

def E(t, r, v):
    t/100
    B = m*v/(q*r)

    b = v/c
    frq = v/r
    phit = frq*t
    phi = lambda x,y: frq*(t - np.sqrt((x - r*np.cos(phit))**2 + (y - r*np.sin(phit))**2)/c)

    bvec = lambda x,y: b * np.array([-np.sin(phi(x,y)), np.cos(phi(x,y))])
    rvec = lambda x,y: np.array([x - r*np.cos(phi(x,y)), y - r*np.sin(phi(x,y))])
    rmag = lambda x,y: np.sqrt((x - r*np.cos(phi(x,y)))**2 + (y - r*np.sin(phi(x,y)))**2)
    rhat = lambda x,y: rvec(x,y)/rmag(x,y)
    diff = lambda x,y: rhat(x,y) - bvec(x,y)

    
    upper = lambda x,y: b*B*k*(diff(x,y)[0]*np.sin(phi(x,y)) - diff(x,y)[0]*np.cos(phi(x,y)))*np.array([rhat(x,y)[1], -rhat(x,y)[0]])
    lower = lambda x,y: c*(1 - b*(rhat(x,y)[1]*np.cos(phi(x,y)) - rhat(x,y)[0]*np.sin(phi(x,y))))**3*rmag(x,y)

    combined = lambda x,y: upper(x,y)/lower(x,y)

    # scaled by distance from origin to minimize the field intensity nearby
    xfunc = lambda x,y: 10*np.sqrt(x**2+y**2)*combined(x,y)[0]
    yfunc = lambda x,y: 10*np.sqrt(x**2+y**2)*combined(x,y)[1]

    F = CustomUV2D(xfunc, yfunc, size=50, resolution=70)
    F.plot()
    
interact(E, t=(0,1,0.001), v=(15e+6, 30e+6), r=(0.1,1,0.01));

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.001), FloatSlider(value=0.55, de…